In [1]:
# import libraries (https://www.kaggle.com/basu369victor/generate-music-with-variational-autoencoder)
import librosa

import numpy as np
import pandas as pd
import os

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import layers 

import matplotlib.pyplot as plt
from IPython import display
from IPython.display import clear_output

import glob
import imageio
import time
import IPython.display as ipd

import music21

AUTOTUNE = tf.data.experimental.AUTOTUNE

In [2]:
# reference: https://hedonistrh.github.io/2018-04-27-Music-Generation-with-LSTM/
midi = music21.converter.parse('./data/muss_1_format0.mid')